In [31]:
from sage.all import *
from random import randrange, shuffle

def miller_rabin(bases, n):
    if n == 2 or n == 3:
        return True

    if n % 2 == 0:
        return False

    r, s = 0, n - 1
    while s % 2 == 0:
        r += 1
        s //= 2
    
    ZmodN = Zmod(n)

    for b in map(ZmodN, bases):
        x = b ** s
        if x == 1 or x == -1:
            continue
        for _ in range(r - 1):
            x *= x 
            if x == -1:
                break
        else:
            return False
    return True

def miller_rabin2(bases, n):
    if n == 2 or n == 3:
        return True

    r, s = 0, n - 1
    while s % 2 == 0:
        r += 1
        s //= 2

    for b in bases:
        x = pow(b, s, n)
        if x == 1 or x == -1:
            continue
        for _ in range(r - 1):
            x = x * x % n
            if x == -1:
                break
        else:
            return False
    return True

prime_list = list(primes(20))+[31337]
print(prime_list)
print(miller_rabin(prime_list, 33))
print(miller_rabin(prime_list, 33))

print(miller_rabin(prime_list, 1234556683))
print(miller_rabin2(prime_list, 1234556683))

[2, 3, 5, 7, 11, 13, 17, 19, 31337]
False
False
True
False


In [26]:
# prime from https://link.springer.com/content/pdf/10.1007%2F978-3-540-30580-4_2
C = 398462957079251*28278016308851*268974870654491*1239515532971*12941222544251*2825874899*182200861571*480965007251*8028415890251*761874633627251*10326412038251*105324823451*7128348371*29542620251*251906132167691*64654312451*226698699371*130685132579*9167201891*432876391197251*3077983389251*17767646051*9371850251*954045342251*112810627931 * 6297653304192251 * 20842025454251

print(miller_rabin(prime_list, C))
print(miller_rabin(prime_list, 2*C+1))

False
True


In [10]:
def gen_random_composite():
    base = list(primes(30)) + [41]
    base.pop(randrange(4,len(base)))
    base.pop(randrange(5,len(base)))
    base += [base[randrange(1,5)]]
    base += [base[randrange(1,5)]]
    base += [base[randrange(1,5)]]
    base += [base[randrange(1,5)]]
    base += [base[randrange(1,5)]]
    base += [base[randrange(1,3)]]
    base += [base[randrange(1,3)]]
    base += [base[randrange(1,3)]]
    base += [base[randrange(1,3)]]
    return base

factorization = []
pseudoprimes = []

while len(pseudoprimes)<5:
    print("Trying to find psuedoprime number ", len(pseudoprimes)+1)
    M = prod(gen_random_composite())
    candidates = [pm1+1 for pm1 in divisors(M) if is_prime(pm1+1) and pm1+1 > 256 and pm1+1 < 2**64]
    print("Trying M = ", M, len(str(M)), " prime candidates: ", len(candidates))
    bins = {}
    for p in candidates:
        rep = 0
        for q in prime_list:
            rep *= 2
            rep += (kronecker(q, p) == 1)
        bins[rep] = bins.get(rep, []) + [p]

    #print([len(bins[k]) for k in bins])
    bins_sorted = [b for b in bins]
    bins_sorted.sort(key=lambda x: -len(bins[x]))
    for index in bins_sorted:
        set_R = bins[index]
        if len(set_R) < 20: break # Not worth even checking small sets
        print("Computing on set of size: ", len(set_R))
        # meet in the middle 
        found = False
        shuffle(set_R)
        SZ = min(20, len(set_R)//2)
        Lhalf = set_R[0:SZ]
        Rhalf = set_R[SZ:2*SZ]

        ppcount = 0
        mid_dict = {}
        for i in range(1, 2**SZ):
            pr = 1
            for j,p in enumerate(Lhalf):
                if i>>j&1:
                    pr = pr*p%M
            if gcd(pr, M) != 1:
                continue
            if pr == 1:
                ppcount +=1
                set_T = [Lhalf[j] for j in range(len(Lhalf)) if i>>j&1]
                pp = prod(set_T)
                if pp < 2**512: continue
                if miller_rabin(prime_list,pp):
                    for ppp in range(2, 55):
                        if miller_rabin(prime_list,ppp*pp+1):
                            pseudoprimes += [[ppp, pp]]
                            factorization += [[ppp]+set_T]
                            print("Adding ", ppp*pp+1)
            mid_dict[pr] = i

        for i in range(1, 2**len(Rhalf)):
            pr = 1
            for j,p in enumerate(Rhalf):
                if i>>j&1:
                    pr = pr*p%M
            if gcd(pr, M) != 1:
                print(pr,[Rhalf[j] for j in range(len(Rhalf)) if i>>j&1], M)
                continue
            invpr = pow(pr, -1, M)
            if invpr in mid_dict or invpr == 1:
                ppcount+=1
                set_T = [Rhalf[j] for j in range(len(Rhalf)) if i>>j&1]
                if invpr !=1:
                    set_T += [Lhalf[j] for j in range(len(Lhalf)) if mid_dict[invpr]>>j&1]
                pp = prod(set_T)
                if pp < 2**512: continue 
                if miller_rabin(prime_list, pp):
                    for ppp in range(2, 55):
                        if miller_rabin(prime_list,ppp*pp+1):
                            pseudoprimes += [[ppp, pp]]
                            factorization += [[ppp]+set_T]
                            print("Adding ", ppp*pp+1)
        print("found pseudoprimes: ", ppcount)
        
print("DONE")

Trying to find psuedoprime number  1
Trying M =  1791912392597250 16  prime candidates:  1129
Computing on set of size:  50
found pseudoprimes:  0
Computing on set of size:  43
found pseudoprimes:  0
Computing on set of size:  40
found pseudoprimes:  0
Computing on set of size:  40
found pseudoprimes:  1
Computing on set of size:  39
found pseudoprimes:  0
Computing on set of size:  39
found pseudoprimes:  0
Computing on set of size:  29
found pseudoprimes:  0
Computing on set of size:  26
found pseudoprimes:  0
Trying to find psuedoprime number  1
Trying M =  36597433309368750 17  prime candidates:  890
Computing on set of size:  24
found pseudoprimes:  0
Computing on set of size:  23
found pseudoprimes:  0
Computing on set of size:  22
found pseudoprimes:  0
Computing on set of size:  21
found pseudoprimes:  1
Computing on set of size:  21
found pseudoprimes:  0
Computing on set of size:  21
found pseudoprimes:  0
Computing on set of size:  21
found pseudoprimes:  0
Computing on set 

found pseudoprimes:  0
Computing on set of size:  29
found pseudoprimes:  0
Computing on set of size:  29
found pseudoprimes:  0
Computing on set of size:  27
found pseudoprimes:  0
Computing on set of size:  21
found pseudoprimes:  0
Trying to find psuedoprime number  4
Trying M =  2271245397671250 16  prime candidates:  1186
Computing on set of size:  47
found pseudoprimes:  0
Computing on set of size:  46
found pseudoprimes:  0
Computing on set of size:  38
found pseudoprimes:  0
Computing on set of size:  36
found pseudoprimes:  0
Computing on set of size:  36
found pseudoprimes:  1
Computing on set of size:  36
found pseudoprimes:  0
Computing on set of size:  35
found pseudoprimes:  0
Computing on set of size:  27
found pseudoprimes:  0
Computing on set of size:  23
found pseudoprimes:  0
Computing on set of size:  22
found pseudoprimes:  0
Computing on set of size:  20
found pseudoprimes:  0
Computing on set of size:  20
found pseudoprimes:  0
Trying to find psuedoprime number  

In [41]:
N = []
for m, pp in pseudoprimes:
    N.append(m*pp+1)
    output = str(pp)
    assert miller_rabin(prime_list, pp)
    assert miller_rabin2(prime_list, pp)
    for prime, mult in factor(m):
        for _ in range(mult):
            output += " " + str(prime)
    print(output)
    print(int(m*pp).bit_length())


363673757901689594566271147214299695717409162156163481609857139957550873529002293761151387224774540113325441715689853618910973083736421022598632113084395001 2 11
522
131411090578999267673271465346122031882151913466048353895451455108023270859765241441232930581063594290979441940931567501362278803741034759879706261458095246937734562722952377803751 2 2 13
601
113217989593673125047319495747009987323060368574281514693214146406574310501245227757206766099977881944509838855920355065723304080190656696486424442660588200192722814284405692768751 2 2 2 2
599
4543008220620359099568524941603313780652613122714195992681692245460735668978306291940477444644525333572260270087760371642558678231686619246262644477169466221251 2 11
535
17200437171172321477295470435135240439117881907148305906802338090064745952591249889387631589470297355668251055968418116638956828571079800041980771417747939032687054793751 2 2 7
567
92320679909974086827268424550757291986176125434240668396077826762848608794103847375829550535890912

In [5]:
# test solve 
from sage.all import *
N = [0 for _ in range(5)]
N[0] = 363673757901689594566271147214299695717409162156163481609857139957550873529002293761151387224774540113325441715689853618910973083736421022598632113084395001 * 2 * 11
N[1] = 131411090578999267673271465346122031882151913466048353895451455108023270859765241441232930581063594290979441940931567501362278803741034759879706261458095246937734562722952377803751 * 2 * 2 * 13
N[2] = 113217989593673125047319495747009987323060368574281514693214146406574310501245227757206766099977881944509838855920355065723304080190656696486424442660588200192722814284405692768751 * 2 * 2 * 2 * 2
N[3] = 4543008220620359099568524941603313780652613122714195992681692245460735668978306291940477444644525333572260270087760371642558678231686619246262644477169466221251 * 2 * 11
N[4] = 17200437171172321477295470435135240439117881907148305906802338090064745952591249889387631589470297355668251055968418116638956828571079800041980771417747939032687054793751 * 2 * 2 * 7
N = [n+1 for n in N]

token = 6976310396091639188523347296072628938811245669053578268918669331148881519157792154445538789225034178622299120196345167772894386165147474204690687781354579264488777004068696894045192616499117512085540160883644383690894844290951164111597016023393298002472294768135113565769465504380361843760844953972780201998
x = [0 for _ in range(5)]
out = [0 for _ in range(5)]

x[0] = 2190745671605969282215177382915450187774829083930977704143519905296682506686951742023702888610602360685779209003050733529864268821432731014887860922173213926
out[0] = 6865388157062047827492731748456995624502570217470094076161357865328942887070569706129332392616339063229905067584158881193451731213909816114918315446999816734
x[1] = 3670649323080732755984349916857272078485915988613425801472904673773491701180673225656000355461700041179663162754988609878816837087314205390147149808757526071039013251602725492616199
out[1] = 4660482633255386103689969932434541801940552783369176818215077929063832825115454949217098545899424934590685051512790091429512282418468888109912511807165783458688589801228582299650847
x[2] = 1015379896338387756609325050236003773432760769367717092421458003103542983516799788819470116390504443697363614824435112298454201661694087612175330318936026859150386629171871112302376
out[2] = 1074494536161426795877538660691334447939156916931740621005665291848673861709790488162406122803310330284831205082629585582120027012399966740978090627763113393056319340770827560394699
x[3] = 58609867604498994073774917308378848551883832970142360193113406440995018859060737047268352044119407774419567439640948164518925887264088392276868454679928319316623
out[3] = 38292315195593782556960486445409254502269687228959176440972024637052273156226656154532910489989659424865764669324533832990784266976384152859748729322715280067686
x[4] = 218732389287748602706172419647405085036487890829191299573932416444337637503549969419282229192146550486242311147978236202612607102328060413001702115503288852203650593041328
out[4] = 268387695466197327268557948526728872974973153595245454399798643502350309347476831199508214962274289883925014980564050087624785401362210313659941075822525266720286404678863

dl = []
mods = []
for i in range(1):
    NN = N[i]
    xx = Integers(NN)(x[i])
    oo = Integers(NN)(out[i])
    dlog = oo.log(xx)%(NN-1)
    print(pow(xx,dlog,NN)==oo)
    for p, mult in factor(N[i]-1):
        if p < 256: continue
        print(dlog%p, p)
        dl.append(dlog%p)
        mods.append(p)
        
        

True
18374 223939
973673 1133731
629545 1154539
3848884 7002451
8166242 8330851
9944580 26729011
40781909 75044971
40043983 78343651
154386163 164390851
633415651 771566251
789536508 996476251
472372351 1110359251
1110583619 2182430251
11863697400 13783259491
23301010959 33831636931
304848329738 359090181451
903176300866 974673349651
1297488161582 2707425971251
True
12532 42751
79429 147799
692075 3228751
7900174 16059751
12241365 18354799
22558288 26114551
265553 42416551
45627051 58315951
58691592 85804951
15794423 95638159
196280349 375554719
541504676 919338751
585187689 2831563351
1562289208 3099384751
4215526741 6332405311
665601291 9312360751
7330838742 14775612391
1555613410 27749320831
16780566802 35332986151
105906091817 124981683751
18062743134 127420350751
True
151756 436591
177806 595351
1082247 1504231
1823506 2037751
1349821 3993991
4025313 10914751
16968966 18288271
33793797 45841951
11707442 50617711
86220060 227130751
257000190 284299471
548361504 1261890631
825988487

In [63]:
ans = crt(dl, mods)
print(ans^token)
from Crypto.Util.number import long_to_bytes
print(long_to_bytes(ans^token))

2529701069150202415145759245888660935698948454718014822883116354342940915912102610953531198507000362037519603585304850716182610938231315800778127873706339271012829908434694336380279513706795940071841884610930876231805
b'uiuctf{w0w_1_th0ughT_Th4t_discr3te_L0g_w4s_h4rD_f04_s4f3_prim2s!1!_d91ea3cf4a3daaf0604520}'
